## Set up experiment

In [1]:
import sys, os; 
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
from src.detr.detr import build_detr
from src.data.dataset import CPPE5Dataset, collate_fn
from src.losses.loss import DETRLoss
from src.utils import pad_targets
from experiment.detr_config import DETRCPPEConfigSanityCheck
from experiment.detr_cppe5_experiment import DETRCPPE5Experiment
from runner.detr_runner import DETRRunner

from torch.utils.data import DataLoader

import torch
import numpy as np
import random

In [3]:
exp = DETRCPPE5Experiment()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
torch.manual_seed(exp.random_seed)
np.random.seed(exp.random_seed)
random.seed(exp.random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(exp.random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
model = build_detr(
    backbone_name= exp.backbone_name,
    num_classes= exp.num_classes,
    num_queries= exp.num_queries,
    d_model=DETRCPPEConfigSanityCheck.d_model,
    nhead=DETRCPPEConfigSanityCheck.nhead,
    num_decoder_layers=DETRCPPEConfigSanityCheck.num_decoder_layers,
    num_encoder_layers=DETRCPPEConfigSanityCheck.num_encoder_layers,
    dim_feedforward=DETRCPPEConfigSanityCheck.dim_feedforward,
    dropout=DETRCPPEConfigSanityCheck.dropout
).to(device)

In [6]:
loss_fn = DETRLoss()
dataset = CPPE5Dataset('../assets/CPPE-5', partition='val')
data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

In [7]:
data_batch = next(iter(data_loader))
images = data_batch['images'].to(device)
masks = data_batch['masks'].to(device)
targets = data_batch['targets']
labels, bboxes = pad_targets(targets, exp.num_queries)

In [8]:
model.train()
output = model(images, masks)
loss_fn(labels.to(device), output[0], bboxes.to(device), output[1])

tensor(255.6964, device='cuda:0', grad_fn=<AddBackward0>)